# ResNet for CIFAR-10

In [1]:
'''ResNet in PyTorch.

For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.linear = nn.Linear(256*block.expansion, num_classes)
        # self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        # self.linear = nn.Linear(512*block.expansion, num_classes)


    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
#         out = self.layer4(out)
#         out = F.avg_pool2d(out, 4)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

    
def ResNet10():
    return ResNet(BasicBlock, [4, 4, 2])


In [2]:
'''Some helper functions for PyTorch, including:
    - get_mean_and_std: calculate the mean and std value of dataset.
    - msr_init: net parameter initialization.
    - progress_bar: progress bar mimic xlua.progress.
'''
import os
import sys
import time
import math

import torch.nn as nn
import torch.nn.init as init


def get_mean_and_std(dataset):
    '''Compute the mean and std value of dataset.'''
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, num_workers=2)
    mean = torch.zeros(3)
    std = torch.zeros(3)
    print('==> Computing mean and std..')
    for inputs, targets in dataloader:
        for i in range(3):
            mean[i] += inputs[:,i,:,:].mean()
            std[i] += inputs[:,i,:,:].std()
    mean.div_(len(dataset))
    std.div_(len(dataset))
    return mean, std

def init_params(net):
    '''Init layer parameters.'''
    for m in net.modules():
        if isinstance(m, nn.Conv2d):
            init.kaiming_normal(m.weight, mode='fan_out')
            if m.bias:
                init.constant(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d):
            init.constant(m.weight, 1)
            init.constant(m.bias, 0)
        elif isinstance(m, nn.Linear):
            init.normal(m.weight, std=1e-3)
            if m.bias:
                init.constant(m.bias, 0)


term_width = 50
                
TOTAL_BAR_LENGTH = 65.
last_time = time.time()
begin_time = last_time
def progress_bar(current, total, msg=None):
    """
    Displays or updates a console progress bar with an arrow indicating progress.

    Args:
    - current: int, the current progress (e.g., the current batch index).
    - total: int, the total count (e.g., the total number of batches).
    - msg: str, additional message to display on the progress bar.
    """
    bar_length = 60
    progress = current / total

    block = int(round(bar_length * progress))
    # Use '>' as an arrow to indicate current progress position
    # Note: The arrow is positioned at the end of the filled portion, except when progress is 0
    arrow = ">" if block < bar_length else ""
    text = "\rProgress: [{0}{1}{2}] {3:.2f}% ({4}/{5}) {6}".format(
        "=" * (block - 1 if block > 0 else 0), arrow,
        "-" * (bar_length - block), progress * 100, current, total,
        msg if msg else "")
    sys.stdout.write(text)
    sys.stdout.flush()
    if current == total - 1:
        sys.stdout.write('\n')

def format_time(seconds):
    days = int(seconds / 3600/24)
    seconds = seconds - days*3600*24
    hours = int(seconds / 3600)
    seconds = seconds - hours*3600
    minutes = int(seconds / 60)
    seconds = seconds - minutes*60
    secondsf = int(seconds)
    seconds = seconds - secondsf
    millis = int(seconds*1000)

    f = ''
    i = 1
    if days > 0:
        f += str(days) + 'D'
        i += 1
    if hours > 0 and i <= 2:
        f += str(hours) + 'h'
        i += 1
    if minutes > 0 and i <= 2:
        f += str(minutes) + 'm'
        i += 1
    if secondsf > 0 and i <= 2:
        f += str(secondsf) + 's'
        i += 1
    if millis > 0 and i <= 2:
        f += str(millis) + 'ms'
        i += 1
    if f == '':
        f = '0ms'
    return f

In [3]:
import zipfile
import pickle

def unpickle(filename):
    zf = zipfile.ZipFile(filename, "r")
    data = pickle.loads(zf.open("cifar_test_nolabels.pkl").read())
    zf.close()
    return data

In [4]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.nn.init as init

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import sys
import time
import math

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
net = ResNet10()


net = net.to(device)
net = torch.nn.DataParallel(net)
if device == 'cuda':
    cudnn.benchmark = True

    
# load state
saved_model = torch.load('./checkpoint/ckpt.pth', map_location=torch.device(device))
net.load_state_dict(saved_model['net'])
net.eval()


/ext3/miniconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..


DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       

In [24]:
import numpy as np

competition_data = unpickle("../data/cifar_test_nolabels.pkl.zip")[b'data']
competition_data = np.reshape(competition_data, (len(competition_data), 32, 32, 3))
competition_data = [transform_test(i) for i in competition_data]
competitionloader = torch.utils.data.DataLoader(
    competition_data, batch_size=1000, shuffle=False)

labels = []

def competition_test():
    net.eval()
    
    with torch.no_grad():
        for batch_idx, input_test in enumerate(competitionloader):
            input_test = input_test.to(device)
            outputs = net(input_test.float())
            _, predicted = outputs.max(1)
            labels.append(predicted)
            print(len(predicted))

            
competition_test()

RuntimeError: The size of tensor a (32) must match the size of tensor b (3) at non-singleton dimension 0

In [14]:
labels

[tensor([1, 9, 0, 1, 0, 0, 8, 2, 9, 2, 8, 9, 9, 8, 8, 8, 3, 0, 9, 8, 7, 9, 8, 8,
         1, 9, 2, 8, 8, 9, 1, 1, 9, 1, 0, 1, 8, 8, 0, 9, 9, 8, 8, 1, 1, 9, 8, 8,
         8, 2, 2, 1, 9, 0, 0, 9, 9, 0, 0, 1, 0, 1, 8, 8, 8, 2, 8, 1, 7, 1, 3, 8,
         1, 3, 9, 1, 1, 8, 0, 8, 0, 8, 8, 2, 8, 8, 1, 0, 0, 9, 7, 1, 2, 1, 1, 8,
         8, 8, 2, 8, 1, 3, 1, 0, 0, 8, 8, 1, 8, 1, 0, 9, 1, 0, 3, 9, 0, 8, 8, 1,
         1, 0, 8, 0, 1, 0, 1, 9, 1, 8, 9, 1, 2, 0, 9, 8, 9, 0, 3, 8, 3, 0, 1, 9,
         1, 8, 0, 9, 0, 2, 8, 0, 9, 8, 9, 1, 9, 9, 0, 1, 1, 1, 1, 8, 9, 8, 8, 8,
         0, 1, 9, 0, 0, 9, 1, 0, 2, 8, 1, 9, 0, 8, 8, 1, 0, 8, 8, 2, 2, 8, 9, 0,
         0, 8, 0, 0, 8, 0, 2, 9, 9, 9, 0, 1, 9, 8, 8, 0, 9, 1, 1, 0, 0, 0, 0, 8,
         9, 8, 7, 8, 8, 0, 8, 8, 1, 0, 8, 9, 0, 2, 0, 3, 1, 8, 9, 0, 9, 8, 9, 2,
         9, 8, 1, 0, 1, 0, 8, 8, 9, 0, 2, 8, 8, 9, 0, 0, 8, 1, 9, 8, 0, 1, 7, 9,
         9, 9, 0, 0, 8, 1, 0, 2, 7, 0, 0, 8, 8, 8, 9, 1, 1, 1, 8, 9, 1, 8, 8, 2,
         9, 1, 0, 9, 8, 9, 1

In [23]:
import pandas as pd

# flatten the labels
pred = np.array([])
for batch in labels:
    pred = np.append(pred, batch.tolist())


competition_data_ids = unpickle("../data/cifar_test_nolabels.pkl.zip")[b'ids']
    
# create dataframe to save prediction
df = pd.DataFrame()
df['ID'] = competition_data_ids
df['Label'] = pred
df['Label'] = df['Label'].astype(int)

df.to_csv('out.csv', index=False)